In [1]:
import pandas as pd
import numpy as np

In [3]:
file_names = ["OD2008_bad.csv", "OD2013_bad.csv", "OD2018_bad.csv"]
features = ['d_orisr', 'd_dessr', 'd_hrede', 'd_mode1']
years = ["2008", "2013", "2018"]
ods = {"2008": None, "2013": None, "2018": None}
convert_dict = {"d_orisr": float, "d_dessr": float}
for year, file_name in zip(years, file_names):
    file = pd.read_csv(file_name)
    file.columns = list(map(str.lower, file.columns))
    file = file[file["p_mobil"]==1]
    file = file[features]
    file.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    file = file.astype(float)
    file.fillna(file.mean(), inplace=True)
    file = file.astype(int)
    a = len(file)
    file = file[file["d_mode1"].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])]
    b = len(file)
    print(b/a*100)
    print(year + " was processed.")
    print("="*15)
    ods[year] = file

99.63369161960188
2008 was processed.
99.56493264612007
2013 was processed.
99.39490997713793
2018 was processed.


In [4]:
oris = {"2008": None, "2013": None, "2018": None}
dess = {"2008": None, "2013": None, "2018": None}

for year in years:
    oris[year] = list(set(list(ods[year]["d_orisr"])))
    oris[year].sort()
    dess[year] = list(set(list(ods[year]["d_dessr"])))
    dess[year].sort()

In [5]:
loc_maps = {"2008": {}, "2013": {}, "2018": {}}
for year in years:
    for i, loc in enumerate(oris[year]):
        loc_maps[year][loc] = str(i+1)

In [6]:
mode_map = {1: "Vehicle", 2: "Vehicle", 3: "Bus", 4: "Subway", 5: "Bus",
            6: "Bus", 7: "Bus",8: "Train", 9: "Bus", 10: "Bus", 11: "Taxi",
            12: "Motorcycle", 13: "Bike", 14: "Walk"}

In [8]:
refined_ods = {"2008": None, "2013": None, "2018": None}
for year in years:
    refined_ods[year] = ods[year][ods[year]["d_hrede"]<=2400]
    refined_ods[year].replace({"d_mode1": mode_map}, inplace=True)
    refined_ods[year].replace({"d_orisr": loc_maps[year]}, inplace=True)
    refined_ods[year].replace({"d_dessr": loc_maps[year]}, inplace=True)
    refined_ods[year]["d_hrede"] = refined_ods[year]["d_hrede"] // 100

c:\program files\python\lib\site-packages\pandas\core\generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)
c:\program files\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
for year in years:
    refined_ods[year].to_csv("OD"+year+"(refined).csv", index=False)

In [13]:
refined_ods["2008"][refined_ods["2008"]["d_mode1"]=="Motorcycle"]

,d_orisr,d_dessr,d_hrede,d_mode1
133,411,101,7,Motorcycle
134,101,411,15,Motorcycle
135,411,409,18,Motorcycle
136,409,411,18,Motorcycle
197,561,561,9,Motorcycle
198,561,561,11,Motorcycle
199,561,540,18,Motorcycle
200,540,561,19,Motorcycle
415,499,509,17,Motorcycle
416,509,499,18,Motorcycle
